---
title: Variance Reduction
description: Advanced techniques for reducing variance in trace estimation including Hutch++ and control variates
keywords: [variance reduction, Hutch++, control variates, low-rank approximation, deflation, adaptive methods, XTrace]
numbering:
  equation:
    enumerator: 6.%s
    continue: true
  proof:theorem:
    enumerator: 6.%s
    continue: true
  proof:algorithm:
    enumerator: 6.%s
    continue: true
  proof:definition:
    enumerator: 6.%s
    continue: true
  proof:proposition:
    enumerator: 6.%s
    continue: true
---

For simplicity, we consider the case when $\vec{A}$ is symmetric positive definite. 
In this case, it is reasonable to ask for a *relative* approximation to the trace, that is, we want an estimate of $\tr(\vec{A})$ that lives in the interval $[(1-\varepsilon)\tr(\vec{A}), (1+\varepsilon)\tr(\vec{A})]$ for some $\varepsilon > 0$.

The variance of the [Girard-Hutchinson estimator](./girard-hutchinson.ipynb#def:girard_hutchinson_estimator) $\widehat{\tr}_m(\cdot)$ scales as $O(1/m)$, where $m$ is the number of samples used. 
For instance, if we use $m$ Gaussian test vectors, then
```{math}
:label: eqn:girard_hutchinson_variance
\EE\left[\widehat{\tr}_m(\vec{A})\right] = \tr(\vec{A})
,\qquad
\VV\left[\widehat{\tr}_m(\vec{A})\right] = \frac{2 \Vert\vec{A}\Vert_\F^2}{m}
\leq \frac{2\tr(\vec{A})^2}{m}.
```
To improve the accuracy of the estimator, we can use variance reduction techniques.


The general idea of modern variance reduction techniques {cite:p}`meyer_musco_musco_woodruff_21,persson_cortinovis_kressner_22,epperly_tropp_webber_24` for implicit estimation is to use a control variate.
The basic idea is simple. 
:::{aside} A note on history
Interestingly, while this approach to variance reduction approach was popularized by Hutch++, the idea is as old as implicit trace estimation itself!
Here is an excerpt from {cite:p}`girard_87`, which may be the first paper on stochastic trace estimation:
![](./girard-variance.png)
:::
Fix any matrix $\vec{B}$ and consider the estimator
```{math}
\widehat{\tr}_m(\vec{A},\vec{B}) := \tr(\vec{B}) + \widehat{\tr}_m(\vec{A} - \vec{B}).
```
This estimator can be implemented so long a we know $\tr(\vec{B})$ and can evaluate the maps $\vec{x} \mapsto \vec{x}^\T\vec{A}\vec{x}$ and $\vec{x} \mapsto \vec{x}^\T\vec{B}\vec{x}$ for any vector $\vec{x}$.
The number of matrix-vector products needed to compute $\widehat{\tr}_m(\vec{A},\vec{B})$ is the same as for $\widehat{\tr}_m(\vec{A})$, namely $m$.
The advantage of the new estimator is that if $\vec{B}$ is a good approximation of $\vec{A}$, then
```{math}
\VV[\widehat{\tr}_m(\vec{A},\vec{B})]
= \VV[\widehat{\tr}_m(\vec{A} - \vec{B})]
```
can be much smaller than $\VV[\widehat{\tr}_m(\vec{A})]$.


## Hutch++

Hutch++ is perhaps the most well-known variance reduction technique for implicit trace estimation.
The idea behind the Hutch++ algorithm is to use a low-rank approximation to $\vec{A}$, computing using the [Randomized SVD](../05-Low-Rank-Approximation/randomized-svd.ipynb) as the control variate. 

The intuition is simple:
- if $\vec{A}$ has a flat spectrum, then $\Vert\vec{A}\Vert_\F \ll \tr(\vec{A})$, and regular Girard-Hutchinson estimator works well.
- if $\vec{A}$ has a decaying spectrum, then it is nearly low-rank, and using a the control variate is helpful.

Below, we present a simple variant of Hutch++ that admits a simple analysis.
The exact variants studied in the original paper differ slightly in the details, but the overall idea is the same.
Readers can also look at Raphael's [Blog Post](https://ram900.com/hutchplusplus/) on Hutch++ for more details. 

:::{prf:algorithm} Hutch++
:label: alg:hutchpp

**Input**: $\vec{A}\in\R^{n\times n}$, samples $m$

1. Sample $\vec{\Omega}\sim\Call{Gaussian}(n,(m+2)/4)$
1. Compute $\vec{Y} = \vec{A}\vec{\Omega}$
1. Compute QR factorization $\vec{Q}\vec{R} = \Call{qr}(\vec{Y})$
1. Compute $\vec{Z} = \vec{A}^\T\vec{Q}$
1. Define $\vec{B} = \vec{Q}\vec{Z}^\T$
1. Compute $\widehat{\tr}_m^{++}(\vec{A}) := \tr(\vec{B}) + \widehat{\tr}_{(m-2)/2}(\vec{A}-\vec{B})$

**Output**: $\widehat{\tr}_m^{++}(\vec{A})$
:::

Observe that Hutch++ requires $m$ matrix-vector products with $\vec{A}$. Indeed, $(m+2)/4$ are used to compute $\vec{A}\vec{\Omega}$, $(m+2)/4$ are used to compute $\vec{Q}^\T\vec{A}$, and $(m-2)/2$ are used to compute $\vec{A}\vec{x}_i$.



Below we implement the Hutch++ estimators. 
Note that by the cyclic property of trace, $\tr(\vec{B}) = \tr(\vec{Q}^\T \vec{A}\vec{Q})$.
This is more computationally efficient to compute, since $\vec{Q}^\T \vec{A}\vec{Q}$ is a small matrix while $\vec{Q}\vec{Q}^\T \vec{A}$ is a large matrix.

In [1]:
def hutchpp(A,m):

    m1 = (m+2)//4
    m2 = m - 2*m1 
    n,_ = A.shape

    Ω = np.random.randn(n,m1)
    Q,_ = np.linalg.qr(A@Ω,mode='reduced')
    Z = A.T@Q
    trB = np.sum(np.diag(Z.T@Q)) 

    X = np.random.randn(n,m2)    
    trm = np.mean(np.diag(X.T@(A@X) - (X.T@Q)@(Z.T@X)))

    return trB + trm

The convergence of Hutch++ compares favorably to {eq}`eqn:girard_hutchinson_variance`.

:::{prf:theorem} 

For any PSD matrix $\vec{A}\in\R^{n\times n}$, if Hutch++ is implemented with Gaussian queries
\begin{equation*}
\EE\left[\widehat{\tr}_m^{++}(\vec{A})\right] = \tr(\vec{A})
,\qquad 
\VV\left[\widehat{\tr}_m^{++}(\vec{A})\right] \leq \frac{16}{(m-2)^2} \tr(\vec{A})^2.
\end{equation*}
:::

:::{prf:proof}
:class: dropdown
:enumerated: false

Let $k+p$ be the number of samples used to sketch $\vec{A}$ in order to build the low-rank approximation, and $\ell$ be the number of samples used to estimate the trace of $\vec{A} - \vec{B}$.

As noted above, $\EE[\widehat{\tr}_m^{++}(\vec{A}) ] = \EE[\EE[\widehat{\tr}_m^{++}(\vec{A}) | \vec{B} ]] = \tr(\vec{A})$ and $\VV[\widehat{\tr}_m^{++}(\vec{A})|\vec{B}] = \VV[\widehat{\tr}_m(\vec{A} - \vec{B})|\vec{B}] = 2\Vert\vec{A} - \vec{B}\Vert_\F^2/\ell$.
Now, by {prf:ref}`thm-rsvd-frob`, a standard bound for the Randomized SVD, we have
```{math}
\EE\left[\Vert\vec{A} - \vec{B}\Vert_\F^2\right]
= \EE\left[\Vert\vec{A} - \vec{Q}\vec{Q}^\T\vec{A}\Vert_\F^2\right]
\leq \left( 1 + \frac{k}{p-1}\right) \Vert \vec{A} - \llbracket \vec{A} \rrbracket_k \Vert_\F^2.
```
As proved in Lemma 7 in {cite:p}`gilbert_strauss_tropp_vershynin_07`,
```{math}
\Vert \vec{A} - \llbracket \vec{A} \rrbracket_k \Vert_\F
\leq \frac{1}{2\sqrt{k}} \tr(\vec{A}).
```
Thus, setting $p=1$,
```{math}
\VV[\widehat{\tr}_m^{++}(\vec{A})|\vec{B}]
\leq \frac{2}{\ell}\left( 1 + \frac{k}{p-1}\right) \frac{1}{4k} \tr(\vec{A})^2
= \frac{1}{\ell k} \tr(\vec{A})^2.
```
Finally, the choices $k = (m-2)/4$ and $\ell = (m-2)/2$ yield the stated bound.

:::

## Numerical Example

Let's set up a numerical experiment to compare Hutch++ to the Girard--Hutchinson estimator.
We'll compare the performance on a problem with fast spectral decay and and one with slow spectral decay. 

In [2]:
import numpy as np
import scipy as sp
from scipy import fft,sparse
import pandas as pd
import time
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from randnla import *

In [3]:
# ========================
# Set up the test problem
# ========================

n = 3000
Λ_fast = 1/np.arange(1,n+1)**2
Λ_slow = 1/np.arange(1,n+1)**1


problems = [
    {'name':'fast decay',
     'Λ':Λ_fast},
    {'name':'slow decay',
     'Λ':Λ_slow}
]

n_ms = 5
ms = np.geomspace(10,1000,n_ms,dtype='int')

methods = [
    {'name':'Girard-Hutchinson',
     'func':girard_hutchinson},
    {'name':'Hutch++',
     'func':hutchpp}
]

# ========================
# Now run the experiment
# ========================
n_repeat = 20
errors = {}
for problem in problems:
    problem_name = problem['name']
    Λ = problem['Λ']
    A = sp.sparse.diags(Λ)
    tr_true = np.sum(Λ)

    for method in methods:

        alg_name = method['name']
        alg = method['func']

        err = np.zeros((n_ms,n_repeat))
        for i,m in enumerate(ms):
            for j in range(n_repeat):

                err[i,j] = np.abs(alg(A,m) - tr_true) / tr_true

        errors[problem_name,alg_name] = err


# ========================
# Plot the results
# ========================
 
σ = .1

fig, axs = plt.subplots(1,2,figsize=(10, 4),sharey=True)

for i,problem in enumerate(problems):

    problem_name = problem['name']
    ax = axs[i]
    ax.set_title(problem_name)
    for method in methods:

        alg_name = method['name']

        err = errors[problem_name,alg_name]

        bot,mid,top = np.quantile(err, [σ, .5, 1-σ], axis=1)

        ax.plot(ms,mid,label=alg_name)
        ax.fill_between(ms,bot,top,alpha=.2)


        ax.set_xlabel('matvecs $m$')
        ax.set_xscale('log')


axs[0].set_ylabel(r'error: $|\operatorname{tr}(A) - \mathrm{alg}|/\operatorname{tr}(A)$')
plt.yscale('log')
plt.legend()

plt.savefig('hutchpp.svg')
plt.close()

The results are as expected. 
When the eigenvalue decay is fast, then Hutch++ performs substantially better than the Girard-Hutchinson estimator since it can effectively remove much of the mass with the low-rank approximation.

![](./hutchpp.svg)

## Variants and Related Methods

While Hutch++ admits a simple analysis, it's clear there is lots of room for modifications/improvements. 
Below we highlight some of the main variants and related works.

### Non-adaptive methods

It's possible to do the variance reduction step in a non-adaptive way; i.e. without multiple passes over $\vec{A}$.
Specifically, the [Randomized SVD](../05-Low-Rank-Approximation/randomized-svd.ipynb) can be replaced with [Generalized Nyström](../05-Low-Rank-Approximation/generalized-nystrom.ipynb).


### Deflation

Several methods have proposed to use compute the top eigencomponents of $\vec{A}$ explicitly and apply a randomized estimator to the residual {cite:p}`weisse_wellein_alvermann_fehske_06,gambhir_stathopoulos_orginos_17,morita_tohyama_20`.

### Adaptive Hutch++

In Hutch++, the number of queries used for low-rank approximation and estimating the trace of the residual are roughly balanced. 
However, there may be better tradeoffs between the two depending on the spectral decay of $\vec{A}$.
In {cite:p}`persson_cortinovis_kressner_22`, the authors propose an adaptive variant of Hutch++ that attempts to use more queries for the low-rank approximation when the spectrum is flat, and more queries for estimating the trace of the residual when the spectrum is decaying.

### XTrace

The XTrace method {cite:p}`epperly_tropp_webber_24` is based on the *exchangability principle* uses all $m$ queries in the same way.
In particular, the method uses all but one query for low-rank approximation, and the last query to estimate the trace of the residual, and then averages this estimate over all possible choices of the last query.
By using a clever low-rank update formula, the algorithm can also be made computationally efficient.
